# Download data
This notebook contains the code to download data from the [Nutritionix API](http://www.nutritionix.com/api) into a reasonable local data structure, create and structure a sqlite3 database, populate the database with data, and save the database to a file on disk.

The restaurant names and IDs have already been downloaded to `restaurant_ids.json`, and this task is beyond the scope of this notebook, so I'm just going to load the restaurant name/ID data from that file.

In [57]:
import requests
import json
import os
import minimum_sugar
import sqlite3
import types

# Load credential data from file
with open("credentials.json", "r") as f:
    credentials = json.load(f)
    
# Load restaurant IDs and names from file
with open("restaurant_ids.json", "r") as f:
    restaurants = json.load(f)

## Get data locally using Nutrionix API
The menu item data needs to be downloaded using the Nutritionix API and stored in a local data structure. This data structure will be a list of dicts, with each list item containing all the data Nutritionix has on that menu item.

In [9]:
# Could do some list comprehension here, but I think it code readability would suffer.
menu_data = []
for restaurant in restaurants:
    menu_data.extend(minimum_sugar.fetch_menu_item_data(restaurant["id"], credentials))

## Create SQLite database
The database needs to be created before it can be populated. The fields will correspond to the data fields provided by the Nutritionix API.

In [77]:
create_table_command_slug = "CREATE TABLE menu_items ({})"

In [78]:
# Map to [SQLite's types](https://www.sqlite.org/datatype3.html)
py2sql_map = {types.NoneType: "text",
    float: "real",
    int: "integer",
    unicode: "text",}

# Create dict with key and sql_key_type mapping
sql_keys_and_types = {key: py2sql_map[type(val)] for key, val in menu_data[0].iteritems()}

# Generate the string used to create the SQL table columns
sql_table_columns_command = ", ".join([" ".join(item) for item in sql_keys_and_types.iteritems()])
create_table_command = create_table_command_slug.format(sql_table_columns_command)

In [80]:
conn = sqlite3.connect("test.sqlite3")
cursor = conn.cursor()
cursor.execute(create_table_command)

In [81]:
conn.close()